In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds 

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10', # dataset name in tensorflow_dataset_catalog
    split=['train', 'test'], # can also have validation
    shuffle_files=True, # multiple tf.records of 1000
                        # even though these every 1000 intenally will be shuffled
                        # we also want ordering of files to shuffle to not see same
                        # order of files multiple times
    as_supervised=True,
    with_info=True,

)



/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteMS62C3/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteMS62C3/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.


#tf.image functions

In [ ]:
def augment(image, label):
  new_height = new_width = 32
  image = tf.image.resize(image, (new_height, new_width))

  if tf.random.uniform((), minval=0, maxval=1) < 0.1:
    image = tf.tile(tf.image.rgb_to_grayscale(image), [1,1,3])
  
  image = tf.image.random_brightness(image, max_delta=0.1)
  image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
  image = tf.image.random_flip_left_right(image)
  return image, label

In [8]:
def normalize_img(image, label):
  # normalize images
  return tf.cast(image, tf.float32)/255.0, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_trian = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(AUTOTUNE)

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(32)
ds_test = ds_test.prefetch(AUTOTUNE)

In [10]:
model = keras.Sequential(
    [
     layers.Input((32, 32, 3)),
     layers.Conv2D(4, 3, padding='same', activation='relu'),
     layers.Conv2D(8, 3, padding='same', activation='relu'),
     layers.MaxPooling2D(),
     layers.Conv2D(16, 3, padding='same', activation='relu'),
     layers.Flatten(),
     layers.Dense(64, activation='relu'),
     layers.Dense(10),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_test, verbose=2)

Epoch 1/5
1563/1563 - 4s - loss: 1.6626 - accuracy: 0.4053
Epoch 2/5
1563/1563 - 4s - loss: 1.3623 - accuracy: 0.5144
Epoch 3/5
1563/1563 - 4s - loss: 1.2325 - accuracy: 0.5649
Epoch 4/5
1563/1563 - 4s - loss: 1.1468 - accuracy: 0.5980
Epoch 5/5
1563/1563 - 4s - loss: 1.0852 - accuracy: 0.6196
313/313 - 2s - loss: 1.1541 - accuracy: 0.5878


[1.1541332006454468, 0.5878000259399414]

#preprocessing layers

In [6]:
data_augmentation = keras.Sequential(
    [
     layers.experimental.preprocessing.Resizing(height=32, width=32),
     layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
     layers.experimental.preprocessing.RandomContrast(factor=0.1)
    ]
)

In [9]:
def normalize_img(image, label):
  # normalize images
  return tf.cast(image, tf.float32)/255.0, label
  
AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(AUTOTUNE)

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(32)
ds_test = ds_test.prefetch(AUTOTUNE)

In [10]:
model = keras.Sequential(
    [
     layers.Input((32,32,3)),
     data_augmentation,
     layers.Conv2D(4, 3, padding='same', activation='relu'),
     layers.Conv2D(8, 3, padding='same', activation='relu'),
     layers.MaxPooling2D(),
     layers.Conv2D(16, 3, padding='same', activation='relu'),
     layers.Flatten(),
     layers.Dense(64, activation='relu'),
     layers.Dense(10),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_test, verbose=2)

Epoch 1/5
1563/1563 - 8s - loss: 1.7400 - accuracy: 0.3773
Epoch 2/5
1563/1563 - 7s - loss: 1.4666 - accuracy: 0.4762
Epoch 3/5
1563/1563 - 7s - loss: 1.3570 - accuracy: 0.5220
Epoch 4/5
1563/1563 - 7s - loss: 1.2802 - accuracy: 0.5476
Epoch 5/5
1563/1563 - 7s - loss: 1.2260 - accuracy: 0.5667
313/313 - 3s - loss: 1.2089 - accuracy: 0.5756


[1.2089338302612305, 0.5756000280380249]